In [1]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib inline

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
from wgand import EnsembleAnomalyDetector
from wgand import load_tissue_graph, get_gdf
from wgand.utils import precision_at_k

/Users/dimakagan/Projects/wgand/wgand/base_detector.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
import networkx as nx
import pickle
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm_notebook as tqdm
from karateclub.node_embedding.neighbourhood import Node2Vec
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [5]:
tissue_path = Path("../data/networks/")
models_path = Path("../models")
out_path = Path("out")

In [6]:
disease_info = pd.read_csv("../data/Disease_Info.csv")
disease_info["Disease_name"] = disease_info["Disease_name"].str.strip("{")
disease_info["Disease_name"] = disease_info["Disease_name"].str.strip("}")
disease_info["Disease_name"] = disease_info["Disease_name"].str.strip("?")
disease_info["Tissue"] = disease_info["Tissue"].str.replace("-", " ")

In [7]:
tissue_mapping = pd.read_csv("../data/tissue_mapping_new.csv")


In [8]:
tissue_mapping = tissue_mapping.dropna()

In [9]:
tissue_mapping = tissue_mapping.merge(pd.DataFrame(disease_info.drop_duplicates(subset=["Tissue","Gene_ID"]).groupby("Tissue").size()), left_on="tissue_name_disease_file", right_on="Tissue", how="left")
tissue_mapping = tissue_mapping.rename(columns={0:"disease_nodes_num"})

In [10]:
tissue_mapping = tissue_mapping[tissue_mapping["disease_nodes_num"]>20]

In [11]:
tissue_mapping = tissue_mapping[~tissue_mapping.tissue_name_network_file.isin(["Breast Mammary Tissue", "Minor Salivary Gland"])]

## Generate Embedding

In [15]:
# from karateclub.node_embedding.neighbourhood import Node2Vec, RandNE
# for _, row in tqdm(tissue_mapping.iterrows()):
#     tissue_name = row["tissue_name_network_file"]
#     tissue_disease_file_name = row["tissue_name_disease_file"]
#     if not (models_path/f"weighted_randne_{tissue_name}.feather").exists():
#         g = load_tissue_graph(tissue_path, tissue_name, tissue_disease_file_name, disease_info)
#         g_num = nx.convert_node_labels_to_integers(g, first_label=0, ordering='sorted', label_attribute="gen")

#         model = RandNE()

#         model.fit(g_num)
        
#         edge_df = nx.convert_matrix.to_pandas_edgelist(g_num)
#         edge_df["source"] = edge_df["source"].astype(int)
#         edge_df["target"] = edge_df["target"].astype(int)
#         edge_df = get_gdf(g)
#         edge_df["features"] = edge_df.apply(lambda x: np.concatenate([model.get_embedding()[int(x["source"])],model.get_embedding()[int(x["target"])]]), axis=1)
#         edge_df.to_feather(models_path/f"weighted_randne_{tissue_name}.feather")

/var/folders/28/wk9vjjr90dngycj11yyzw4nw0000gn/T/ipykernel_23947/1320421976.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _, row in tqdm(tissue_mapping.iterrows()):


0it [00:00, ?it/s]

In [13]:
emb_model = "RandNE"
if (models_path/f"{emb_model}.pkl").exists():
    print("Loading embeddings")
    edge_df = pd.read_pickle(models_path/f"{emb_model}.pkl")
else:
    edge_df = nx.convert_matrix.to_pandas_edgelist(g_num)
    edge_df["source"] = edge_df["source"].astype(int)
    edge_df["target"] = edge_df["target"].astype(int)
    edge_df = get_gdf(g)
    edge_df["features"] = edge_df.apply(lambda x: np.concatenate([model.get_embedding()[int(x["source"])],model.get_embedding()[int(x["target"])]]), axis=1)
    edge_df.to_pickle(models_path/f"{emb_model}.pkl")

Loading embeddings


# Ours

## Node Embeddings

In [16]:
from karateclub.node_embedding.neighbourhood import RandNE, GLEE, NodeSketch, DeepWalk
node_emb_models = [RandNE, GLEE, NodeSketch, DeepWalk, Node2Vec]

In [ ]:
from pyod.models.iforest import IForest
import pyod.models.combination as comb 
res = []
df_concat = []

with tqdm(total=(len(node_emb_models)*len(tissue_mapping))) as pbar:

    for model_cls in node_emb_models:
        edge_df = pd.read_pickle(models_path/f"{model_cls.__name__}.pkl")

        for _, row in tissue_mapping.iterrows():
            tissue_name = row["tissue_name_network_file"]
            tissue_name_disease_file = row["tissue_name_disease_file"]
            temp = {"tissue_name":tissue_name, "embedding":model_cls.__name__}
            print(f"---------{tissue_name}-{tissue_name_disease_file}--------")
            try:
                g = load_tissue_graph(tissue_path, tissue_name, tissue_name_disease_file, disease_info)

                nodes = list(g.nodes)
                y = [1 if "disease_name" in g.nodes[n] else 0 for n in nodes ]
            except KeyError:
                print(f"{tissue_name} not found in disease info")
                continue
            nad = EnsembleAnomalyDetector(g, RandomForestRegressor(n_jobs=-1,n_estimators=500,random_state=2), IForest(n_jobs=-1,random_state=2))
            nad.set_gdf_features(edge_df["features"])
            nad.fit(nodes)
        
            probs = nad.predict_node_proba(nodes)

            temp["auc"] = roc_auc_score(y, probs)
            temp["p@1"] = precision_at_k(y, probs,1)
            temp["p@3"] = precision_at_k(y, probs,3)
            temp["p@10"] = precision_at_k(y, probs,10)
            temp["p@20"] = precision_at_k(y, probs,20)
            temp["p@anom"] = precision_at_k(y, probs,np.sum(y))
            temp["anom"] = np.sum(y)
            res.append(temp.copy())  
            pbar.update(1)

In [19]:
pd.DataFrame(res).to_csv(out_path/"embedding-results.csv")

In [18]:
pd.DataFrame(res).groupby(["embedding"]).mean()

,auc,p@1,p@3,p@10,p@20,p@anom,anom
embedding,,,,,,,
DeepWalk,0.662935,0.411765,0.235294,0.194118,0.158824,0.094793,142.941176
GLEE,0.669870,0.352941,0.254902,0.176471,0.141176,0.089198,142.941176
Node2Vec,0.665810,0.411765,0.274510,0.241176,0.182353,0.097830,142.941176
NodeSketch,0.669975,0.411765,0.313725,0.247059,0.194118,0.096789,142.941176
RandNE,0.670086,0.529412,0.372549,0.252941,0.214706,0.099098,142.941176


# Weight Estimator

In [16]:
from pyod.models.iforest import IForest
import pyod.models.combination as comb 
res = []
df_concat = []
link_clfs = [
            (xgb.XGBRegressor, {"n_jobs":-1,"n_estimators": 100,"max_depth":9, "subsample":0.8,"eta":0.5 ,"seed":2, "eval_metric":"rmse","colsample_bytree":1}),
            (lgb.LGBMRegressor, {"n_estimators": 100,"random_state":2, "learning_rate":0.1, "num_leaves":512, "n_jobs":-1}),
            (RandomForestRegressor, {"n_jobs":-1,"n_estimators": 500,"random_state":2})
            ] 

with tqdm(total=(len(link_clfs)*len(tissue_mapping))) as pbar:

    for clf, link_clf_params in link_clfs:
        for _, row in tissue_mapping.iterrows():
            tissue_name = row["tissue_name_network_file"]
            tissue_name_disease_file = row["tissue_name_disease_file"]
            temp = {"tissue_name":tissue_name, "Estimator":clf.__name__}
            link_clf = clf(**link_clf_params)
            print(f"---------{tissue_name}-{tissue_name_disease_file}--------")
            try:
                g = load_tissue_graph(tissue_path, tissue_name, tissue_name_disease_file, disease_info)

                nodes = list(g.nodes)
                y = [1 if "disease_name" in g.nodes[n] else 0 for n in nodes ]
            except KeyError:
                print(f"{tissue_name} not found in disease info")
                continue
            nad = EnsembleAnomalyDetector(g, link_clf, IForest(n_jobs=-1,random_state=2))
            nad.set_gdf_features(edge_df["features"])
            nad.fit(nodes)
        
            probs = nad.predict_node_proba(nodes)

            temp["auc"] = roc_auc_score(y, probs)
            temp["p@1"] = precision_at_k(y, probs,1)
            temp["p@3"] = precision_at_k(y, probs,3)
            temp["p@10"] = precision_at_k(y, probs,10)
            temp["p@20"] = precision_at_k(y, probs,20)
            temp["p@anom"] = precision_at_k(y, probs,np.sum(y))
            temp["anom"] = np.sum(y)
            res.append(temp.copy())  
            pbar.update(1)

  0%|          | 0/51 [00:00<?, ?it/s]

---------Artery Aorta-Artery Aorta--------
---------Brain Cerebellum-Brain Cerebellum--------
---------Brain Cortex-Brain Cortex--------
---------Brain Spinal cord cervical c 1-Spinal cord--------
---------Heart Atrial Appendage-Heart Atrium--------
---------Heart Left Ventricle-Heart Ventricle--------
---------Liver-Liver--------
---------Lung-Respiratory system Lung--------
---------Muscle Skeletal-Skeletal muscle--------
---------Nerve Tibial-Peripheral nerve--------
---------Ovary-Ovary--------
---------Pituitary-Pituitary--------
---------Skin Not Sun Exposed Suprapubic-Skin--------
---------Skin Sun Exposed Lower leg-Skin--------
---------Testis-Testis--------
---------Whole Blood-Blood and bone marrow--------
---------Whole Brain-Brain--------
---------Artery Aorta-Artery Aorta--------
---------Brain Cerebellum-Brain Cerebellum--------
---------Brain Cortex-Brain Cortex--------
---------Brain Spinal cord cervical c 1-Spinal cord--------
---------Heart Atrial Appendage-Heart Atri

In [17]:
pd.DataFrame(res).groupby(["Estimator"]).mean()

,auc,p@1,p@3,p@10,p@20,p@anom,anom
Estimator,,,,,,,
LGBMRegressor,0.644254,0.176471,0.137255,0.123529,0.082353,0.056497,142.941176
RandomForestRegressor,0.670087,0.529412,0.372549,0.252941,0.214706,0.099098,142.941176
XGBRegressor,0.602994,0.000000,0.000000,0.011765,0.023529,0.028443,142.941176


In [18]:
pd.DataFrame(res).to_csv(out_path/"link-clf-results.csv")